In [38]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# === Load saved scaler parameters ===
scaler_mean = np.load("scaler_mean_combined.npy")
scaler_scale = np.load("scaler_scale_combined.npy")

# Recreate the scaler
scaler = StandardScaler()
scaler.mean_ = scaler_mean
scaler.scale_ = scaler_scale
scaler.var_ = scaler_scale ** 2

from tensorflow.keras.models import load_model
model = load_model("final_combined_cnn_model.h5")

In [39]:
import pandas as pd

data_BBH_df = pd.read_csv("./BBH/processed.csv", header=None)
X = data_BBH_df.values.astype(float)

print(len(data_BBH_df.index))

y = [1 for i in range(len(data_BBH_df.index))]

X = scaler.transform(X)

X = np.expand_dims(X, axis=-1)
print(X.shape)

48
(48, 16384, 1)


In [40]:
pred = model.predict(X)
y_pred = (pred.ravel() > 0.5).astype(int)
correct = np.sum(y_pred == y)
acc = (correct / len(y)) * 100

print(f"Total no. of original gravitational data samples: {len(y)}")
print(f"No. of correct predictions: {correct}")
print(f"Accuracy: {acc:.2f}%")

2/2 [==============================] - 1s 278ms/step
Total no. of original gravitational data samples: 48
No. of correct predictions: 41
Accuracy: 85.42%


In [41]:
import pandas as pd

data_BBH_df = pd.read_csv("./BBH/original-noise-unseen.csv", header=None)
X = data_BBH_df.values.astype(float)

print(len(data_BBH_df.index))

y = [0 for i in range(len(data_BBH_df.index))]

X = scaler.transform(X)

X = np.expand_dims(X, axis=-1)
print(X.shape)

48
(48, 16384, 1)


In [42]:
pred = model.predict(X)
y_pred = (pred.ravel() > 0.5).astype(int)
correct = np.sum(y_pred == y)
acc = (correct / len(y)) * 100

print(f"Total no. of original noise data samples: {len(y)}")
print(f"No. of correct predictions: {correct}")
print(f"Accuracy: {acc:.2f}%")

2/2 [==============================] - 0s 236ms/step
Total no. of original noise data samples: 48
No. of correct predictions: 48
Accuracy: 100.00%
